# Exercise 1: Compute pi on the GPU

## Tasks

1. Write a CPU-program that computes pi using geometry and statistics
2. Port this program to the GPU
3. Optimize

## Solution

<-- see `compute_pi_solution.ipynb`

In [ ]:
import numpy as np
import time

rng = np.random.default_rng()

def compute_pi_cpu(n_points):
    x_rand = rng.random(n_points)
    y_rand = rng.random(n_points)

    inside = np.sqrt(x_rand**2 + y_rand**2) <= 1.0
    n_inside = np.sum(inside)

    #n_inside = 0
    #for i in range(n_points):
    #    n_inside += np.sqrt(x_rand[i]**2 + y_rand[i]**2) <= 1.0

    pi = 4*n_inside/n_points

    return pi

In [ ]:
%%time

tic = time.time()
print(compute_pi_cpu(100000))
toc = time.time()

print("Time for CPU-version: {:f} seconds".format(toc-tic))

3.13044
Time for CPU-version: 0.003916 seconds
CPU times: total: 0 ns
Wall time: 4.92 ms


In [1]:
pi_kernel_src = """

//Based on Stroustrup, adapted for CUDA
//pseudorandom numbers
__device__ float generateRandomNumber(long& last_draw) {
    last_draw = last_draw*1103515245 + 12345;
    long abs = last_draw & 0x7fffffff;
    return abs / 2147483648.0;
}

__global__ void computePi(unsigned int* inside, unsigned int seed) {
  unsigned int tid = threadIdx.x;

  long rand_seed = seed + tid;
  float x = generateRandomNumber(rand_seed);
  float y = generateRandomNumber(rand_seed);

  float r = sqrt(x*x + y*y);

  if(r <= 1.0) {
    inside[tid] = 1;
  }
}
"""

mod = SourceModule(pi_kernel_src)
func = mod.get_funtion("computePi")

In [ ]:
def compute_pi_gpu(n_points):
  bytes_per_uint = 4

  inside_gpu = cuda.mem_alloc(bytes_per_uint * n_points)

